<a href="https://colab.research.google.com/github/NH0917/petfinder/blob/main/swin_and_augmix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
no = "PetFineder"
description = "ベンチーマーク_v3にAugmixを追加"
#swin_large_patch4_window7_224

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.simplefilter('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install timm
!pip install albumentations==0.4.6
!pip install transformers
!pip install mlflow
!pip install ttach

In [3]:
from albumentations.augmentations.transforms import ShiftScaleRotate
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import albumentations as A
import cv2
from tqdm.notebook import tqdm
import  matplotlib.pylab as plt
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import StratifiedKFold
import timm
import os
from albumentations.pytorch import ToTensorV2
import gc
import mlflow
import shutil
import torch.optim as optim
from torch.optim import optimizer
from torch.optim import lr_scheduler
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import ttach as tta
from pandas.core.algorithms import value_counts
import torchvision
from PIL import Image, ImageOps, ImageEnhance



---



In [4]:
def int_parameter(level, maxval):
    """Helper function to scale `val` between 0 and maxval .
    Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
    Returns:
    An int that results from scaling `maxval` according to `level`.
    """
    return int(level * maxval / 10)


def float_parameter(level, maxval):
    """Helper function to scale `val` between 0 and maxval.
    Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
    Returns:
    A float that results from scaling `maxval` according to `level`.
    """
    return float(level) * maxval / 10.


def sample_level(n):
    return np.random.uniform(low=0.1, high=n)


def autocontrast(pil_img, _):
    return ImageOps.autocontrast(pil_img)


def equalize(pil_img, _):
    return ImageOps.equalize(pil_img)


def posterize(pil_img, level):
    level = int_parameter(sample_level(level), 4)
    return ImageOps.posterize(pil_img, 4 - level)


def rotate(pil_img, level):
    degrees = int_parameter(sample_level(level), 30)
    if np.random.uniform() > 0.5:
        degrees = -degrees
    return pil_img.rotate(degrees, resample=Image.BILINEAR)


def solarize(pil_img, level):
    level = int_parameter(sample_level(level), 256)
    return ImageOps.solarize(pil_img, 256 - level)


def shear_x(pil_img, level):
    level = float_parameter(sample_level(level), 0.3)
    if np.random.uniform() > 0.5:
        level = -level
    return pil_img.transform(pil_img.size,
                           Image.AFFINE, (1, level, 0, 0, 1, 0),
                           resample=Image.BILINEAR)


def shear_y(pil_img, level):
    level = float_parameter(sample_level(level), 0.3)
    if np.random.uniform() > 0.5:
        level = -level
    return pil_img.transform(pil_img.size,
                           Image.AFFINE, (1, 0, 0, level, 1, 0),
                           resample=Image.BILINEAR)


def translate_x(pil_img, level):
    level = int_parameter(sample_level(level), pil_img.size[0] / 3)
    if np.random.random() > 0.5:
        level = -level
    return pil_img.transform(pil_img.size,
                           Image.AFFINE, (1, 0, level, 0, 1, 0),
                           resample=Image.BILINEAR)


def translate_y(pil_img, level):
    level = int_parameter(sample_level(level), pil_img.size[0] / 3)
    if np.random.random() > 0.5:
        level = -level
    return pil_img.transform(pil_img.size,
                           Image.AFFINE, (1, 0, 0, 0, 1, level),
                           resample=Image.BILINEAR)


# operation that overlaps with ImageNet-C's test set
def color(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Color(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def contrast(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Contrast(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def brightness(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Brightness(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def sharpness(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Sharpness(pil_img).enhance(level)


augmentations = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y
]

augmentations_all = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y, color, contrast, brightness, sharpness
]

def normalize(image):
    """Normalize input image channel-wise to zero mean and unit variance."""
    return image - 127

def apply_op(image, op, severity):
    #   image = np.clip(image, 0, 255)
    pil_img = Image.fromarray(image)  # Convert to PIL.Image
    pil_img = op(pil_img, severity)
    return np.asarray(pil_img)

def aug(image, preprocess,mixture_width,mixture_depth,aug_severity):
  """Perform AugMix augmentations and compute mixture.
  Args:
    image: PIL.Image input image
    preprocess: Preprocessing function which should return a torch tensor.
  Returns:
    mixed: Augmented and mixed image.
  """
  aug_list = augmentations
  
  ws = np.float32(np.random.dirichlet([1] * mixture_width))
  m = np.float32(np.random.beta(1, 1))

  
  mix = torch.zeros_like(preprocess(image=image)["image"])
  for i in range(mixture_width):
    image_aug = image.copy().astype(np.uint8)
    depth = mixture_depth if mixture_depth > 0 else np.random.randint(
        1, 4)
    for _ in range(depth):
      op = np.random.choice(aug_list)
      image_aug = apply_op(image_aug,op,aug_severity)

    # Preprocessing commutes since all coefficients are convex
    mix += ws[i] * preprocess(image=image_aug)["image"]

  mixed = (1 - m) * preprocess(image=image)["image"] + m * mix
  return mixed

class AugMixDataset(torch.utils.data.Dataset):
  """Dataset wrapper to perform AugMix augmentation."""

  def __init__(self,dataset,preprocess, no_jsd=False,mixture_width=3,mixture_depth=10,aug_severity=10):
    self.dataset = dataset
    self.preprocess = preprocess
    self.no_jsd = no_jsd
    self.mixture_width = mixture_width
    self.mixture_depth = mixture_depth
    self.aug_severity = aug_severity


  def __getitem__(self, i):
    
    img,label = self.dataset[i]
  
    if self.no_jsd:
      return aug(img, self.preprocess,self.mixture_width,self.mixture_depth,self.aug_severity), label
    else:
      im_tuple = (self.preprocess(image=img)["image"], aug(img, self.preprocess),
                  aug(img, self.preprocess))
  
      return im_tuple, label

  def __len__(self):
      return len(self.dataset)


class petfinder(Dataset):
    def __init__(self,df,data_dir,feature_col,transforms=False):
        self.df = df
        self.data_dir = data_dir
        self.feature_col = feature_col
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        img_name = self.df["Id"][idx]
        img_path = os.path.join(self.data_dir,img_name) + str(".jpg")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

        label = self.df["Pawpularity"][idx]
        feature = self.df[self.feature_col].loc[idx].values

        if self.transforms:
            img = self.transforms(image=img)
            img = img["image"]

            #img = img/255

            #img = torch.tensor(img,dtype=torch.float)

        else:
            img
        return img,label



In [5]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [6]:
#CFG
def seed_everything(seed=42):
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

CFG = {
    "IMG_SIZE":384,
    "batch_size":8,
    "LR":1e-5,
    "epochs":50,
    "patience":2,
    "MODEL_NAME":"swin_large_patch4_window12_384_in22k",
    "pretrained":True,
    "device":"cuda",
    "nfolds":10,
    "grad_accum_steps":0,
    "use_amp":True,
    "debug":False
}

train_augmentation = A.Compose([
                                 A.HorizontalFlip(p=0.5),  
])

test_augmentation = A.Compose([
    A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"]),
    A.Normalize([0.485,0.456,0.406],[0.229,00.224,0.225]),
    ToTensorV2()
])

preprocess = test_augmentation

In [7]:
#Read data and define path

train_path = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/input/train_skf10.csv"
data_dir = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/input/train"
save_dir = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/output"
mlflow_dir = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/input/mlflow"

train = pd.read_csv(train_path)
train["Pawpularity"] = train["Pawpularity"]/100


feature_col = [col for col in train.columns.tolist() if col not in ["Id","Pawpularity","fold"]]

In [ ]:
#train and eval

def loss_fn(y_hat,y_true):
    y_hat = torch.sigmoid(y_hat) * 100
    y_true = y_true*100
    return torch.sqrt(torch.mean((y_hat-y_true)**2))

class PetModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.model = timm.create_model(CFG["MODEL_NAME"],pretrained=CFG["pretrained"])
        self.model.head = nn.Linear(1536,1)
        
    def forward(self,x):
        x = self.model(x)
        return x.squeeze()

def train_fn(train_progress_bar,model,loss_fn,device,optimizer,lr_scheduler,):
    loss = 0
    metrics = AverageMeter()
    criterion = torch.nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler()
    model.train()

    for i,(img,labels) in enumerate(train_progress_bar):
  
        optimizer.zero_grad()
        img = img.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with torch.cuda.amp.autocast(enabled=True):
            outputs = model(img)
            loss = criterion(outputs,labels)
            rmse = loss_fn(outputs,labels)

        metrics.update(rmse.item(),batch_size)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()

    return metrics.avg

def val_fn(val_progress_bar,model,loss_fn,device):
    metrics = AverageMeter()
    model.eval()
    preds = []

    for i ,(img,labels) in enumerate(val_progress_bar):
      img = img.to(device)
      labels = labels.to(device)
      batch_size = labels.size(0)

      with torch.no_grad():
        outputs = model(img)
      rmse = loss_fn(outputs,labels)
      metrics.update(rmse.item(),batch_size)
      outputs = torch.sigmoid(outputs) * 100
      preds.append(outputs.cpu().numpy())

      predictions = np.concatenate(preds)

    return metrics.avg,predictions

def train_loop(train_dl,val_dl,fold,cfg,val_df):
      train_progress_bar = tqdm(train_dl)
      val_progress_bar = tqdm(val_dl)
      counter = 0

      model = PetModel(cfg)
      model.to(cfg["device"])
      
      optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5)
      lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=10*len(train_dl))
      
      for epoch in range(cfg["epochs"]):
          print(f"{epoch}")

          train_rmse = train_fn(train_progress_bar,model,loss_fn,cfg["device"],optimizer,lr_scheduler)
          val_rmse,predict = val_fn(val_progress_bar,model,loss_fn,cfg["device"])

          print(f"train_rmse is {train_rmse}")
          print(f"val_rmse is {val_rmse}")


          if epoch == 0 or best_metric > val_rmse:
              print("This epoch is best metric")
              if os.path.exists(save_dir):
                  pass
              else:
                  os.mkdir(save_dir)
              print("Save model")
              torch.save(model.state_dict(),os.path.join(save_dir,f"model_state_fold_{fold}.pth"))
              best_metric = val_rmse
              best_train_rmse = train_rmse

              val_df["predict"] = predict
              val_df.to_csv(os.path.join(save_dir,f"result_fold{fold}.csv"),index=False)
              print("Save df")


          else:
              print(f"Best metric == {best_metric:.6f} This Epoch metric == {val_rmse:.6f} So not Saving")
              counter += 1
              print(f"Counter is {counter}")

          if counter == CFG["patience"]:
              print("Early Stopping")
              break
    
      
      return best_train_rmse,best_metric
    

def main(CFG):
  
  train_rmse_list = []
  val_rmse_list = []

  for fold in range(CFG["nfolds"]):

      print(f"-"*100)
      print(f"Start Fold{fold}")
      print(f"UseCol{feature_col}")
        
      train_idx,val_idx = train.query(f"fold!={fold}").index,train.query(f"fold=={fold}").index
      train_df,val_df = train.loc[train_idx].reset_index(drop=True),train.loc[val_idx].reset_index(drop=True)
      
      if CFG["debug"]:
        print("This seccions is debug mode")
        train_df = train_df.loc[:100]
        val_df = val_df.loc[:100]
        CFG["epochs"] = 1

      
      train_ds = petfinder(train_df,data_dir,feature_col,transforms=train_augmentation)
      train_ds = AugMixDataset(train_ds, preprocess,True)
      train_dl = DataLoader(train_ds,batch_size=CFG["batch_size"],shuffle=True,num_workers=2,pin_memory=True,drop_last=True)
      val_ds = petfinder(val_df,data_dir,feature_col,transforms=preprocess)
      val_dl = DataLoader(val_ds,batch_size=CFG["batch_size"]*2,shuffle=False,num_workers=2,pin_memory=True,drop_last=False)

      train_rmse,val_rmse = train_loop(train_dl,val_dl,fold,CFG,val_df)
      train_rmse_list.append(train_rmse)
      val_rmse_list.append(val_rmse)

      print(f"End Fold{fold}")
      print(f"-"*100)

  return train_rmse_list,val_rmse_list

def ploting_and_output(total_loss_and_acc_list,save_dir):
    
    metrics_name = ["train_loss","valid_loss"]

    for i,metric in enumerate(metrics_name):
        
        fig,ax = plt.subplots()

        ax.set_xlabel("Epoch")
        ax.set_xlabel(metric)

        for j,metric_num in enumerate(total_loss_and_acc_list[i]):
            ax.plot(range(len(metric_num)),metric_num,label="Fold_{}".format(j))
        plt.legend() 
        fig.savefig(os.path.join(save_dir,"{}.png".format(metric)))

def save_mlflow(mlflow_dir,save_dir,total_loss_and_acc_list,cfg,desc,no):
    
    print("-"*100)
    dic1 = dict()
    dic2 = dict()
    train_acc_list_tmp = []
    val_acc_list_tmp = []
    train_acc_list,val_acc_list = total_loss_and_acc_list

    for i,metric in enumerate(train_acc_list):
        dic1[f"train_rmse_fold{i}"] = metric
    mean_acc = np.mean(train_acc_list)
    dic2["train_rmse_mean"] = mean_acc
    
    for i,metric in enumerate(val_acc_list):
        dic1[f"val_rmse_fold{i}"] = metric
    mean_acc = np.mean(val_acc_list)
    dic2["val_rmse_mean"] = mean_acc
        
    mlflow.set_tracking_uri(mlflow_dir)
    mlflow.set_experiment(no)
    
    with mlflow.start_run():
        mlflow.log_params(cfg)
        mlflow.log_metrics(dic1)
        mlflow.log_metrics(dic2)
        mlflow.log_artifact(save_dir)

    print("save mlflow")
    print("-"*100)

if __name__ == "__main__":
    total_loss_and_acc_list = main(CFG)
    #ploting_and_output(total_loss_and_acc_list,save_dir)
    save_mlflow(mlflow_dir,save_dir,total_loss_and_acc_list,CFG,description,no)
    #shutil.copyfile("train.py",os.path.join(save_dir,train.py))

----------------------------------------------------------------------------------------------------
Start Fold0
UseCol['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']


  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


0
train_rmse is 17.97657904636585
val_rmse is 16.76389574243885
This epoch is best metric
Save model
Save df
1
train_rmse is 17.097702233792006
val_rmse is 16.63940547182849
This epoch is best metric
Save model
Save df
2
train_rmse is 16.46861728116061
val_rmse is 16.60896768795037
This epoch is best metric
Save model
Save df
3
train_rmse is 15.412548155408158
val_rmse is 16.676613635745277
Best metric == 16.608968 This Epoch metric == 16.676614 So not Saving
Counter is 1
4
